# Baseline

Creates paraphrases based on a word-for-word replacement with synonyms from the [CJVT Thesaurus of Modern Slovene](http://hdl.handle.net/11356/1166).

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, treebank
import os
import pickle as pkl
from tqdm import tqdm

In [2]:
dict_loc = "data/thesaurus_dict.pkl"

if os.path.exists(dict_loc) and os.path.isdir(dict_loc):
    with open(dict_loc, "rb") as f:
        thesaurus = pkl.load(f)
else:
    with open("data/CJVT_Thesaurus-v1.0/CJVT_Thesaurus-v1.0.xml") as f:
        xml_doc = f.read()
        soup = BeautifulSoup(xml_doc, 'xml')
    thesaurus = dict()
    for entry in tqdm(soup.find_all("entry"), desc="loop over entries"):
        if entry.groups_core: # TODO: add some threshold based on score in candidate xml tags
            candidate = entry.groups_core.group.find("candidate")
            if float(candidate["score"]) >= 0.16:
                thesaurus[str(entry.headword.string)] = str(candidate.s.string)
    with open(dict_loc, "wb") as f:
        pkl.dump(thesaurus, f)

print("Number of all synonyms in the dictionary:", len(thesaurus))

loop over entries: 100%|██████████| 105473/105473 [00:04<00:00, 25705.01it/s]

Number of all synonyms in the dictionary: 81297


In [3]:
df = pd.read_pickle("data/4th_test.pkl").df

In [4]:
# optional quick check of dataset
#for _, (c1, c2) in df.iterrows():
#    print(c1, "\n", c2)
#    break

In [5]:
for _, (orig, backt) in list(df.iterrows())[:2]: # TODO: remove list(...)[:2] to get all paraphrases
    new_stt = []
    for token in word_tokenize(orig):
        if token in thesaurus:
            new_stt.append(thesaurus[token])
        else:
            new_stt.append(token)
    print(f"ORIG: {orig}\nBACKT: {backt}\nBASE: {treebank.TreebankWordDetokenizer().detokenize(new_stt)}\n")

ORIG: Za razvojno perspektivo učenja jezika (semantike, skladnje in pragmatike) ni pomembno le učenje slovničnih pravil, ampak so pomembne vse vrste višjih duševnih procesov, še posebej učenje konceptov oz. pojmov.
BACKT: Za razvojno perspektivo učenja jezikov (semantika, sintaksa in pragmatika) ni pomembno le, da se naučimo slovničnih pravil, temveč so pomembne vse vrste višjih miselnih procesov, zlasti učenje konceptov.
BASE: Za razvojno perspektivo učenja jezika (semantike, skladnje in pragmatike) ni važno samo uk slovničnih pravil, ampak so pomembne vse krogi višjih duševnih procesov, še posebej uk konceptov oz . pojmov.

ORIG: Velike temperaturne obremenitve in visoki tlaki so zahtevali svoj davek in taki motorji so bili precej občutljivi in dokaj zahtevni za vzdrževanje. Velik problem je bila tudi "turbo luknja" oziroma pomanjkanje moči pri nižjih motornih vrtljajih. Kot kaže so pri Volkswagnu našli rešitev za vsakdanje potrebe. Združili so svojo turbo tehnologijo in neposredni v